# 数据预处理

In [1]:
import numpy as np                                        # 导入numpy库并简写为np
from sklearn import datasets                              # 导入datasets模块，用于加载鸢尾花的数据集

iris_dataset = datasets.load_iris()                       # 加载鸢尾花的数据集，并存在iris_dataset

In [2]:
X = iris_dataset.data[:100, :].astype(np.float32)         # 选取iris_dataset的data的前100个数据，将其数据类型转换为float32，并储存在X中
X_feature_names = iris_dataset.feature_names              # 将iris_dataset的特征名称储存在X_feature_names中
y = iris_dataset.target[:100].astype(int)                 # 选取iris_dataset的target的前100个数据，将其数据类型转换为int，并储存在y中
y_target_names = iris_dataset.target_names[:2]            # 选取iris_dataset的target_names的前2个数据，并储存在y_target_names中


In [3]:
alpha = X[:, :3] * X[:, 1:]           # 每一个样本中，利用相邻两个特征值计算出一个参数，即每一个样本会多出3个参数（因为有4个特征值），并储存在alpha中
X = np.append(X, alpha, axis=1)       # 在axis=1的维度上，将alpha的数据值添加到X的特征值中

print(X.shape)                        # 打印此时X的样本的数据维度

(100, 7)


In [4]:
def feature_normalize(data):
    mu = np.mean(data,axis=0)
    std = np.std(data,axis=0)
    return (data - mu)/std
X = feature_normalize(X)

In [5]:
from sklearn.model_selection import train_test_split                                                   # 导入train_test_split函数，用于对数据集进行划分

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, shuffle=True) # 将数据集划分为训练集和测试集
print(X_train.shape)                                                                                   # 打印训练集中样本的数据类型
print(X_test.shape)
print(y_test)


(80, 7)
(20, 7)
[0 1 0 1 1 1 0 1 1 1 1 1 1 0 0 0 0 0 0 0]


In [6]:
# sigmoid function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

## 数据归一化

## 3.3 将程序提交到量子计算云平台(同QCIS流程)<a id="jump_3_3"></a>

将编译出的QCIS线路提交到量子计算云平台。

In [7]:
qcis_circuit = '''
    H    Q1
    H    Q8
    H    Q14
    H    Q21
   RZ    Q1 {alpha0}
   RZ    Q8 {alpha1}
   RZ    Q14 {alpha2}
   RZ    Q21 {alpha3}

  Y2M    Q8
   CZ    Q1           Q8
  Y2P    Q8

   RZ    Q8 {alpha4}

  Y2M    Q8
   CZ    Q1           Q8
  Y2P    Q8

  Y2M    Q14
   CZ    Q8           Q14
  Y2P    Q14
   RZ    Q14 {alpha5}
  Y2M    Q14
   CZ    Q8           Q14
  Y2P    Q14

  Y2M    Q21
   CZ    Q14           Q21
  Y2P    Q21
   RZ    Q21 {alpha6}
  Y2M    Q21
   CZ    Q14           Q21
  Y2P    Q21

   RY    Q1 {theta0}
   RY    Q1 {theta1}
   RY    Q1 {theta2}
   RY    Q1 {theta3}

   Y2M   Q8
   CZ    Q1          Q8
   Y2P   Q8

   Y2M   Q14
   CZ    Q8          Q14
   Y2P   Q14

   Y2M   Q21
   CZ    Q14          Q21
   Y2P   Q21

   RY    Q1 {theta4}
   RY    Q1 {theta5}
   RY    Q1 {theta6}
   RY    Q1 {theta7}
   Z Q14
   Z Q21
   M Q14 
   M Q21
   '''

In [22]:
from ezQpy import * 
from math import pi
# account = Account(login_key='9d60894fd6692bd00ee529e6bddd8d31', machine_name='应答机A')
account = Account(login_key="9d60894fd6692bd00ee529e6bddd8d31", machine_name="ClosedBetaQC")
#设置用户SDK密钥，选择量子计算机，并创建实例，运行成功后，提示登录成功。

lab_id = "c9329ae3fb6b4862aa5f62edbdf98c98"
# lab_id = 'b744756a208f487ebf04a54a686d9ad1'
# exp_id = '393b52d6c3774918a7e418ab0082e11a'
# exp_id = None

In [23]:
# create_res = account.create_experiment('test')
# if create_res == 0:
#     print('新建实验合集失败')
# else :
#     print('新建实验合辑成功，ID=', create_res)
#     lab_id = create_res
# #实验合集id建议长期保留，以便后继使用。

exp_id = account.save_experiment(lab_id, qcis_circuit, version='1.0')

保存实验失败：失败


In [15]:
exp_id

## Todo:
1. 梯度计算
$\frac{\partial L}{\partial \theta_i} = \frac{<B>^+_i - <B>^-_i}{2}$
2. $\theta$ 的更新
$\theta_{j+1} = \theta_j - \alpha \frac{\partial L}{\partial \theta_j}$
3. loss定义
$loss = -\sum_{i=1}^m y_i\log(\hat{y})$

In [32]:
import time
# train
eta = 0.1 #学习率
loss = 0
theta = [1 for i in range(8)]
for j in range(5):
    alpha = X_train[j].tolist()
    params = alpha + theta
    for i in range(8):
        params[i+7] += pi/2

        qcis_circuit_with_params = account.assign_parameters(circuit=qcis_circuit, parameters=['alpha0', 'alpha1', 'alpha2', 'alpha3', 'alpha4', 'alpha5', 'alpha6', 'theta0', 'theta1', 'theta2', 'theta3', 'theta4', 'theta5', 'theta6', 'theta7'], values=params)
        time_str = time.strftime("%Y%m%d%H%M%S", time.localtime())
        exp_id = account.save_experiment(lab_id, qcis_circuit_with_params, version=time_str + str(j))
        query_id=account.run_experiment(exp_id=exp_id, num_shots=1000)#num_shots在上述submit_job中采用了默认值12000。
        prob_1 = 0

        if query_id:
            result=account.query_experiment(query_id, max_wait_time=360000)
            #最大等待时间单位为秒，不传递时默认为30秒。因量子程序的执行会有排队的情况，而量子计算机本身有自动校准的时间，如果想跑全自动的程序，等待时间最好大于两者。
            prob_1 = account.readout_data_to_state_probabilities_whole(result)['10']+account.readout_data_to_state_probabilities_whole(result)['11']
            #后继数据应用实现。
            B_plus = sigmoid(prob_1)
            
            
        params[i+7] -= pi
        time_str = time.strftime("%Y%m%d%H%M%S", time.localtime())
        qcis_circuit_with_params = account.assign_parameters(circuit=qcis_circuit, parameters=['alpha0', 'alpha1', 'alpha2', 'alpha3', 'alpha4', 'alpha5', 'alpha6', 'theta0', 'theta1', 'theta2', 'theta3', 'theta4', 'theta5', 'theta6', 'theta7'], values=params)
        exp_id = account.save_experiment(lab_id, qcis_circuit_with_params, version=time_str + str(j))
        query_id=account.run_experiment(exp_id=exp_id, num_shots=1000)#num_shots在上述submit_job中采用了默认值12000。
        prob_2 = 0
        if query_id:
            result=account.query_experiment(query_id, max_wait_time=360000)
            #最大等待时间单位为秒，不传递时默认为30秒。因量子程序的执行会有排队的情况，而量子计算机本身有自动校准的时间，如果想跑全自动的程序，等待时间最好大于两者。
            prob_2 = account.readout_data_to_state_probabilities_whole(result)['10']+account.readout_data_to_state_probabilities_whole(result)['11']
            #后继数据应用实现  
            B_minus = sigmoid(prob_1)
        gradient = (prob_1 - prob_2)/2
        theta[i] = theta[i] - eta * gradient

查询实验结果请等待: 2.46秒
查询实验结果请等待: 3.08秒
查询实验结果请等待: 4.17秒
查询实验结果请等待: 1.56秒
查询实验结果请等待: 4.96秒
查询实验结果请等待: 3.06秒
查询实验结果请等待: 3.63秒
查询实验结果请等待: 3.24秒
查询实验结果请等待: 2.57秒
查询实验结果请等待: 2.45秒
查询实验结果请等待: 4.31秒
查询实验结果请等待: 5.39秒
查询实验结果请等待: 3.82秒
查询实验结果请等待: 2.37秒
查询实验结果请等待: 1.54秒
查询实验结果请等待: 3.70秒
查询实验结果请等待: 3.70秒
查询实验结果请等待: 3.46秒
查询实验结果请等待: 2.23秒
查询实验结果请等待: 1.46秒
查询实验结果请等待: 3.31秒
查询实验结果请等待: 4.56秒
查询实验结果请等待: 4.94秒
查询实验结果请等待: 4.43秒
查询实验结果请等待: 3.76秒
查询实验结果请等待: 4.93秒
查询实验结果请等待: 5.82秒
查询实验结果请等待: 1.58秒
查询实验结果请等待: 1.64秒
查询实验结果请等待: 2.37秒
查询实验结果请等待: 4.53秒
查询实验结果请等待: 1.12秒
查询实验结果请等待: 1.68秒
查询实验结果请等待: 4.93秒
查询实验结果请等待: 1.47秒
查询实验结果请等待: 2.19秒
查询实验结果请等待: 2.49秒
查询实验结果请等待: 0.62秒
查询实验结果请等待: 2.11秒
查询实验结果请等待: 3.05秒
查询实验结果请等待: 4.94秒
查询实验结果请等待: 4.36秒
查询实验结果请等待: 2.97秒
查询实验结果请等待: 1.60秒
查询实验结果请等待: 4.95秒
查询实验结果请等待: 3.31秒
查询实验结果请等待: 1.72秒
查询实验结果请等待: 2.55秒
查询实验结果请等待: 0.64秒
查询实验结果请等待: 4.36秒
查询实验结果请等待: 4.51秒
查询实验结果请等待: 0.71秒
查询实验结果请等待: 3.54秒
查询实验结果请等待: 2.67秒
查询实验结果请等待: 5.18秒
查询实验结果请等待: 3.41秒
查询实验结果请等待: 4.01秒
查询实验结果请等待: 1.30秒
查询实验结果请等待: 2.2

In [33]:
alpha, theta

([-1.0509300231933594,
  -0.20783503353595734,
  -1.0129777193069458,
  -0.8642762303352356,
  -0.860920250415802,
  -1.109249472618103,
  -0.9215091466903687],
 [1.0015500000000002,
  1.0019,
  0.9991500000000001,
  0.9963,
  1.0029500000000002,
  0.9991000000000001,
  1.0022999999999997,
  0.9995])

In [34]:
correct = 0
for j in range(20):
    alpha = X_test[j].tolist()
    params = alpha + theta
    for i in range(8):
        params[i+7] += pi/2

        qcis_circuit_with_params = account.assign_parameters(circuit=qcis_circuit, parameters=['alpha0', 'alpha1', 'alpha2', 'alpha3', 'alpha4', 'alpha5', 'alpha6', 'theta0', 'theta1', 'theta2', 'theta3', 'theta4', 'theta5', 'theta6', 'theta7'], values=params)
        query_id=account.run_experiment(exp_id=exp_id, num_shots=12000)#num_shots在上述submit_job中采用了默认值12000。

        if query_id:
            result=account.query_experiment(query_id, max_wait_time=360000)
            #最大等待时间单位为秒，不传递时默认为30秒。因量子程序的执行会有排队的情况，而量子计算机本身有自动校准的时间，如果想跑全自动的程序，等待时间最好大于两者。
            prob_1 = account.readout_data_to_state_probabilities_whole(result)['10']+account.readout_data_to_state_probabilities_whole(result)['11']
            prob_2 = account.readout_data_to_state_probabilities_whole(result)['00']+account.readout_data_to_state_probabilities_whole(result)['01']
        if prob_1 > prob_2 and y_test[j] == 0:
            correct += 1
        if prob_1 > prob_2 and y_test[j] == 1:
            correct += 1
print(correct/20)

查询实验结果请等待: 1.76秒
查询实验结果请等待: 3.39秒
查询实验结果请等待: 0.64秒
查询实验结果请等待: 0.48秒
查询实验结果请等待: 3.63秒
查询实验结果请等待: 2.92秒
查询实验结果请等待: 0.17秒
查询实验结果请等待: 1.87秒
查询实验结果请等待: 1.51秒
查询实验结果请等待: 1.42秒
查询实验结果请等待: 0.79秒
查询实验结果请等待: 4.82秒
查询实验结果请等待: 4.91秒
查询实验结果请等待: 1.04秒
查询实验结果请等待: 2.15秒
查询实验结果请等待: 2.40秒
查询实验结果请等待: 4.05秒
查询实验结果请等待: 1.69秒
查询实验结果请等待: 4.87秒
查询实验结果请等待: 3.88秒
查询实验结果请等待: 5.42秒
查询实验结果请等待: 5.53秒
查询实验结果请等待: 4.58秒
查询实验结果请等待: 0.91秒
查询实验结果请等待: 0.27秒
查询实验结果请等待: 3.13秒
查询实验结果请等待: 3.28秒
查询实验结果请等待: 4.53秒
查询实验结果请等待: 5.53秒
查询实验结果请等待: 4.94秒
查询实验结果请等待: 4.67秒
查询实验结果请等待: 2.66秒
查询实验结果请等待: 0.98秒
查询实验结果请等待: 1.98秒
查询实验结果请等待: 2.07秒
查询实验结果请等待: 0.91秒
查询实验结果请等待: 1.97秒
查询实验结果请等待: 3.80秒
查询实验结果请等待: 3.23秒
查询实验结果请等待: 0.54秒
查询实验结果请等待: 3.50秒
查询实验结果请等待: 4.78秒
查询实验结果请等待: 3.22秒
查询实验结果请等待: 1.44秒
查询实验结果请等待: 3.35秒
查询实验结果请等待: 2.05秒
查询实验结果请等待: 4.44秒
查询实验结果请等待: 1.23秒
查询实验结果请等待: 2.35秒
查询实验结果请等待: 0.99秒
查询实验结果请等待: 2.46秒
查询实验结果请等待: 5.67秒
查询实验结果请等待: 5.91秒
查询实验结果请等待: 0.46秒
查询实验结果请等待: 3.96秒
查询实验结果请等待: 4.95秒
查询实验结果请等待: 5.37秒
查询实验结果请等待: 1.86秒
查询实验结果请等待: 3.3